In [123]:
import pandas as pd
import os
import numpy as np
from gensim.models import Word2Vec, FastText
from mittens import GloVe

# import glove
# from glove import Corpus

import collections
import gc 

import keras
from keras import backend as K
from keras import regularizers
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Dropout, Input, concatenate, merge, Activation, Concatenate, LSTM, GRU
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Conv1D, BatchNormalization, GRU, Convolution1D, LSTM
from keras.layers import UpSampling1D, MaxPooling1D, GlobalMaxPooling1D, GlobalAveragePooling1D,MaxPool1D, merge

from keras.optimizers import Adam


from keras.utils import np_utils
from keras.backend.tensorflow_backend import set_session, clear_session, get_session
import tensorflow as tf

from keras.callbacks import EarlyStopping, ModelCheckpoint, History, ReduceLROnPlateau


from sklearn.utils import class_weight
from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score, f1_score

import warnings
warnings.filterwarnings('ignore')


from keras.models import load_model
from keras.utils import CustomObjectScope
from keras.initializers import glorot_uniform

In [124]:
## Load Data ##

type_of_ner = "new"

x_train_lstm = pd.read_pickle("data/"+type_of_ner+"_x_train.pkl")
x_dev_lstm = pd.read_pickle("data/"+type_of_ner+"_x_dev.pkl")
x_test_lstm = pd.read_pickle("data/"+type_of_ner+"_x_test.pkl")

y_train = pd.read_pickle("data/"+type_of_ner+"_y_train.pkl")
y_dev = pd.read_pickle("data/"+type_of_ner+"_y_dev.pkl")
y_test = pd.read_pickle("data/"+type_of_ner+"_y_test.pkl")

In [134]:
timeseries_model_results = [{"modelName":"GRU-256-los_3-best_model.hdf5", "results":"results/256-GRU-los_3-10-new.p", "problem":"los_3"},{"modelName":"GRU-256-los_7-best_model.hdf5", "results":"results/256-GRU-los_7-10-new.p", "problem":"los_7"},{"modelName":"GRU-256-mort_hosp-best_model.hdf5", "results":"results/256-GRU-mort_hosp-10-new.p", "problem":"mort_hosp"},{"modelName":"GRU-256-mort_icu-best_model.hdf5", "results":"results/256-GRU-mort_icu-10-new.p", "problem":"mort_icu"},{"modelName":"LSTM-256-los_3-best_model.hdf5", "results":"results/256-LSTM-los_3-10-new.p", "problem":"los_3"},{"modelName":"LSTM-256-los_7-best_model.hdf5", "results":"results/256-LSTM-los_7-10-new.p", "problem":"los_7"},{"modelName":"LSTM-256-mort_hosp-best_model.hdf5", "results":"results/256-LSTM-mort_hosp-10-new.p", "problem":"mort_hosp"},{"modelName":"LSTM-256-mort_icu-best_model.hdf5", "results":"results/256-LSTM-mort_icu-10-new.p", "problem":"mort_icu"}]

In [135]:
def make_prediction_timeseries(model, test_data):
    probs = model.predict(test_data)
    y_pred = [1 if i>=0.5 else 0 for i in probs]
    return probs, y_pred

def printComparison(modelName, resultsFile, model_output):
    stats = pd.read_pickle(resultsFile)
    print("--------------------------")
    print(f"modelName: {modelName}, auc: {model_output['auc']==stats['auc']}, auprc: {model_output['auprc']==stats['auprc']}, acc: {model_output['acc']==stats['acc']} F1: {model_output['F1']==stats['F1']}")
    print(stats)

def get_model_output(problem, probs, predictions):
    ground_truth = y_test[problem].values
    auc = roc_auc_score(ground_truth, probs)
    auprc = average_precision_score(ground_truth, probs)
    acc   = accuracy_score(ground_truth, predictions)
    F1    = f1_score(ground_truth, predictions)
    model_output = {}    
    model_output['auc'] = auc
    model_output['auprc'] = auprc
    model_output['acc'] = acc
    model_output['F1'] = F1
    return model_output
    

In [136]:
## loop through each model and print comparision results ##
for model_data in timeseries_model_results:
    model = load_model(model_data['modelName'], custom_objects={'_initializer': glorot_uniform()})
    probs, predictions = make_prediction_timeseries(model, x_test_lstm)
    model_output = get_model_output(model_data['problem'], probs, predictions)
    printComparison(model_data['modelName'], model_data['results'], model_output)
    

--------------------------
modelName: GRU-256-los_3-best_model.hdf5, auc: True, auprc: True, acc: True F1: True
{'auc': 0.6893080498763451, 'auprc': 0.6273247606110519, 'acc': 0.656163753449862, 'F1': 0.5358584290592984}
--------------------------
modelName: GRU-256-los_7-best_model.hdf5, auc: True, auprc: True, acc: True F1: True
{'auc': 0.7242005077570872, 'auprc': 0.20549892835355515, 'acc': 0.9174333026678932, 'F1': 0.021798365122615803}
--------------------------
modelName: GRU-256-mort_hosp-best_model.hdf5, auc: True, auprc: True, acc: True F1: True
{'auc': 0.8739894484095516, 'auprc': 0.5576770818641951, 'acc': 0.9139834406623735, 'F1': 0.42813455657492355}
--------------------------
modelName: GRU-256-mort_icu-best_model.hdf5, auc: True, auprc: True, acc: True F1: True
{'auc': 0.8771797993039626, 'auprc': 0.4832968288657088, 'acc': 0.9362925482980681, 'F1': 0.3661327231121282}
--------------------------
modelName: LSTM-256-los_3-best_model.hdf5, auc: True, auprc: True, acc: Tru